# Last left off
- Change experimental_vars list:
    - move the GPU list up to the experiment_setup JSON object (below counts of GPUs)... also get it to validate against which GPUs it actually ran on
    - experiment_setup: move the GPU count and type above the CPU count and type
- ASK O3 TO CONTINUE
- CHECK QUERY RATE AND DECODER TEMP AND OTHER FUNCTIONALITY JUST ADDED
    - GPU list: doesn't run on different GPUs (somewhere it's hard coded to 01)
    - decoder temp: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.

# Next
- [ ] make some changes between ExperimentConfig and ExperimentRunner
    - [ ] take the "inference_fn" of the notebook_launcher and instead have it determined in the run_experiment flow, by the parameter "purely_generative" of the experiment_config object.
    - [ ] take the `backend` parameter from the ExperimentRunner and (i) have it determined in the experiment_config class, and (ii) outputed in the hardware setup output of the JSON
- [ ] validate `tracking_mode="machine"` in `EmissionsTracker`
- [ ] move some of the experiment utils to metrics,py 
    - rename metrics_results.py
    - rename function `aggregate_experiments` to `aggregate_experiment_results`
- [ ] set up AMD CPU logic (currently only INTEL) -> confirm CPU usage metric
- [ ] try CodeCarbon's save_to_logger 
- [ ] distributed env issues
    - error logs:
        - Launching training on 2 GPUs.
        - Using device: cuda:0 (Local Rank: 0)
        - Model is on cuda:1
        - [codecarbon WARNING @ 18:23:07] Multiple instances of codecarbon are allowed to run at the same time.
        - Using 2 GPUs: [0, 1]
        - Model is on cuda:0
        - [codecarbon WARNING @ 18:23:07] Multiple instances of codecarbon are allowed to run at the same time.
        - Aggregated benchmark results saved to benchmark_results/text_generation_results.json
        - [2025-03-05 18:23:17,500] torch.distributed.elastic.multiprocessing.api: [WARNING] Closing process 3902871 via signal SIGTERM
    - solution
        - It looks like there's some inconsistency in how the GPUs are being assigned. Based on the logs:The training is being launched on 2 GPUs ([0, 1]). The device at Local Rank: 0 is cuda:0, but at a later point, the model is reported on cuda:1 and then cuda:0 again. codecarbon is running multiple instances, which might indicate multiple runs being initiated in parallel.The training process (PID 3902871) was closed via SIGTERM, which could be due to an error, manual interruption, or a resource issue.
        - Check GPU Device Assignment: Ensure that your training script correctly assigns GPUs without switching unexpectedly. If using torch.nn.DataParallel, it should handle this automatically.
        - If using torch.nn.parallel.DistributedDataParallel (DDP), check that each process is assigned the correct GPU using:
            """python
            Copy
            Edit
            local_rank = int(os.environ["LOCAL_RANK"])
            torch.cuda.set_device(local_rank)
            If using manual device placement, explicitly check model.to(device) to avoid conflicts.
            """
- refactor: (i) helper functions; (ii) Object oriented
- from run_gen_inf() remove line `do_sample = True if inference_type == "reasoning" else False`
- validate if the ExperimentConfigs actually change anything

## Review Metrics:
- [ ] review compute metrics:
    - remove cpu vendor 
    - remove cuda 
    - sort out memory allocation metrics (they repeat, are they ever different?)
    - sort out the gpu_utilisation_percent <- currently it shows overall utilisation by other programmes, but I think that is right? check?
- [ ] work out what the GPU and CPU power metrics coming out of CodeCarbon are referring to (avg power draw?)
- [ ] GPU & CPU power = 'avg across processors' <- add this in (ram power constant)
- [ ] GPU & CPU energy = sum across processort
- [ ] NB the RAM energy seems to be wrong (I am summing, but RAM is shared ????)
    - NB THIS IS A MAJOR POINT: CODE CARBON GETS IT WRON BY THE PROCESS BY NOT CONSIDERING SHARED RESOURCES...??? 
- [ ] FLOPs monitoring
- [ ] to experimental_variables: add avg_input_tokens and avg_output_tokens between and within batches / across the overall experiment??
- is it possible to record power everysec as a file --> allow for plotting over time to see how this changes with (?)
    
- [ ] if staying on Hertie server: build in AMD tracking tools for the CPU (currently it can't measure it using CC)
- [ ] log metrics both at (1) the individual per prompt, (2) the per-batch level, and (3) the experiment level (e.g. number of input/output tokens, energy etc etc)

# Blockers
- [ ] vLLM -> CUDA runs out of memory
- [ ] Optimum benchmarking library -> CUDA runs out of memory

# Low priority
- [ ] add Carbon Tracker
- [ ] add logic for task-specific metrics in the reporting
- [ ] how is final GPU / CPU power calculated by CodeCarbon? is it an avg?
- [ ] sort out the print statements in experiments to supress (i) the code carbon output, (ii) the final JSON print

# Done
- [x] change codeCarbon log_level to warning 
- [x] Code Carbon turn off multiple runs in allow_multiple_runs
- [x] calculate computation metrics
- [x] put the text_gen_inference() into the helper functionn (as it is pretty generic)
- [x] us Optimum library 
- [x] sort out distributed inference using accelerate / vLLM. At the moment it's not working on either
- [x] aggregate process metrics 
    - once it goes to multi-GPU it gets weird - i think this is why half the runs don't work, the other half do
    - including assinging the unique ID
    - decide which metrics to sum, which to avg
- [x] check why the CodeCarbon CSV metrics and the JSON metrics don't align - Answer: because they are per process -> allows me to validate (also check that CodeCarbon don't offer their own aggregation)
    - CC can do by machine or by process
        - parameter: tracking_mode -> machine measure the power consumptions of the entire machine (defaults) / process try and isolate the tracked processes in isolation